In [ ]:
import sys
import json

# sys.path.insert(0,'/groups/dso/meijie/melanoma_data/Code_Bahadir/Wispermed/extraction_funs')
# sys.path.insert(0,'/groups/dso/meijie/melanoma_data/fhir_data/Top2000Melanomexport-2022-04-30T10:49:34')
# sys.path.insert(0,'/groups/dso/meijie/melanoma_data/fhir_data')
sys.path.insert(0,'extraction_funs')

In [ ]:
from extraction_baseclass import extractor, tryPath

In [ ]:
careplanExtractor = extractor(name='CarePlan', resourceType='CarePlan') # Create an instance of baseclass for careplan
careplanExtractor.addField('patid', lambda x: x['subject']['reference'], True) # throws an error if doesnt exist
careplanExtractor.addField('dt_start', lambda x: tryPath(x, ['period','start']),False) # will return none if it doesnt exist

In [ ]:
conds = {
    'cat_therapy_aim': lambda x: tryPath(x, ['coding',0,'system']) == 'https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation/TherapyPlanning/Therapieziel',
    'cat_therapy_type': lambda x: tryPath(x, ['coding',0,'system']) == 'https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation/TherapyPlanning/Therapie'
}
careplanExtractor.addFieldsFromList(['category','*','coding',0,'display'], conds)
# About careplan extractor we need both a path and conditions. Because path only would take everything with that path with condition we get what we want.


#### So far we added fields to the careplanextractor instance using method from the baseclass. We also added fields from list w.r.t specific condition to get what we want. This was just to create a careplan extractor demo. For this instance, we must add as many fields as needed (i.e, we take every relevant field from careplans). Now we can call the patient json data and test our careplanextractor object.

In [ ]:
#json_pat = open('/groups/dso/meijie/melanoma_data/fhir_data/0a1ea535edd2820fd0c6d652f26a744bee5a52ce6055a6d993fbee0e065a724e.json') # pick a patient
json_pat = open('data_new/0a1ea535edd2820fd0c6d652f26a744bee5a52ce6055a6d993fbee0e065a724e.json')
json_pat = json.load(json_pat) # load the patient data

In [ ]:
careplans_transformed = careplanExtractor.extract(json_pat["entry"]) # Now use the object and create a filtered patient json file.
careplans_transformed 

In [ ]:
# It looks and searchs for the care plan and takes all things inside
careplans = [x['resource'] for x in json_pat["entry"] if x['resource']['resourceType'] == "CarePlan"]

# Now we take all the careplans entries for the patient json file. Now we have both a careplan extractor at hand and the careplans of the patient

#careplans2 = []
#for x in json_pat["entry"]:
#    if x['resource']['resourceType'] == "CarePlan":
#        careplans2.append(x['resource'])

#careplans[0:2] # Unlock this to see if careplans really has careplans :)

In [ ]:
from medication_extractor import *
from tnm_extractors import getExtractorProgress

In [ ]:
getMedicationAdministration().extract(json_pat['entry'])

In [ ]:
progresses = getExtractorProgress().extract(json_pat['entry'])
#progresses[0:5] # Unlock to see output

In [ ]:
from group_and_merge import groupResources, combineResources
# group resources gives us all resources with the same date in a list:
groupedProgress = groupResources(progresses, "dt_record")
groupedProgress[0]

In [ ]:
# groupedProgress[1] # Unlock to see output

In [ ]:
# we now combine the grouped resources:
combinedResources = [combineResources(x) for x in groupedProgress]

In [ ]:
# combinedResources[0] # Unlock to see output

In [ ]:
# groupedProgress[1] # Unlock to see output

In [ ]:
# changes from local